In [5]:
import pandas as pd
import numpy as np
from pid_sm import PIDController
from concurrent.futures import ProcessPoolExecutor

# Function to simulate the system response using the PID controller
def simulate_system_response(pid_controller, setpoint, object_centers):
    data = {'Time': [], 'Input': [], 'Output': []}

    for object_center in object_centers:
        error = setpoint - object_center
        output = pid_controller.update(error)

        # Simulate time increments of 0.5 seconds
        time_increment = 0.5
        data['Time'].append(len(data['Time']) * time_increment)
        data['Input'].append(error)
        data['Output'].append(output)

    return pd.DataFrame(data)

def evaluate_parameters(kp, ki, kd, setpoint, object_centers):
    pid_controller = PIDController(kP=kp, kI=ki, kD=kd)
    pid_controller.reset()
    df = simulate_system_response(pid_controller, setpoint, object_centers)
    total_error = np.sum(np.abs(df['Output']))
    return {'kP': kp, 'kI': ki, 'kD': kd, 'Error': total_error}

# Generate input values
object_centers = np.arange(-320, 321, 10)

# Setpoint for all cases
setpoint = 320

# Define the ranges for tuning parameters
kp_values = np.linspace(0.1, 2.0, 20)
ki_values = np.linspace(0.01, 0.5, 10)
kd_values = np.linspace(0.0, 0.5, 10)

# Best tuning parameters
best_params = None
best_error = float('inf')

# List to store close tuning parameter sets
close_params_list = []

# Close threshold for parallelization
close_threshold = 50  # Adjust as needed

# Function to check if the error is close to the best error
def is_close(error):
    return error < best_error + close_threshold

# Using ProcessPoolExecutor for parallelization
with ProcessPoolExecutor() as executor:
    # Iterate through possible combinations of kp, ki, and kd
    futures = []
    for kp in kp_values:
        for ki in ki_values:
            for kd in kd_values:
                futures.append(executor.submit(evaluate_parameters, kp, ki, kd, setpoint, object_centers))

    # Retrieve results from futures
    for future in futures:
        params = future.result()
        total_error = params['Error']

        # Check if the current parameters result in a better response
        if total_error < best_error:
            best_error = total_error
            best_params = {'kP': params['kP'], 'kI': params['kI'], 'kD': params['kD']}

        # Check if the error is close to the best error
        if is_close(total_error):
            close_params_list.append({'kP': params['kP'], 'kI': params['kI'], 'kD': params['kD'], 'Error': total_error})

# Output the best tuning parameters
print("Best Tuning Parameters:")
print(best_params)

# Save best_params to an Excel file using DataFrame
best_params_df = pd.DataFrame([best_params])
best_params_df.to_excel('best_tuning_params.xlsx', index=False)

# Print the head of the DataFrame
print("\nHead of the DataFrame:")
print(best_params_df.head())


Best Tuning Parameters:
{'kP': 0.1, 'kI': 0.01, 'kD': 0.05555555555555555}

Head of the DataFrame:
    kP    kI        kD
0  0.1  0.01  0.055556
